In [1]:
# ch06 随机森林
# 步骤1: 创建SparkSession对象
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('random_forest').getOrCreate()

In [2]:
# 步骤2: 读取数据集
df = spark.read.csv('affairs.csv', inferSchema=True, header=True)

In [3]:
print(df.count(), len(df.columns))

6366 6


In [4]:
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)



In [6]:
df.show(5)


+-------------+----+-----------+--------+---------+-------+
|rate_marriage| age|yrs_married|children|religious|affairs|
+-------------+----+-----------+--------+---------+-------+
|            5|32.0|        6.0|     1.0|        3|      0|
|            4|22.0|        2.5|     0.0|        2|      0|
|            3|32.0|        9.0|     3.0|        3|      1|
|            3|27.0|       13.0|     3.0|        1|      1|
|            4|22.0|        2.5|     0.0|        1|      1|
+-------------+----+-----------+--------+---------+-------+
only showing top 5 rows



In [7]:
df.describe().select('summary','rate_marriage', 'age', 'yrs_married', 'children', 'religious').show()

+-------+------------------+------------------+-----------------+------------------+------------------+
|summary|     rate_marriage|               age|      yrs_married|          children|         religious|
+-------+------------------+------------------+-----------------+------------------+------------------+
|  count|              6366|              6366|             6366|              6366|              6366|
|   mean| 4.109644989004084|29.082862079798932| 9.00942507068803|1.3968740182218033|2.4261702796104303|
| stddev|0.9614295945655025| 6.847881883668817|7.280119972766412| 1.433470828560344|0.8783688402641785|
|    min|                 1|              17.5|              0.5|               0.0|                 1|
|    max|                 5|              42.0|             23.0|               5.5|                 4|
+-------+------------------+------------------+-----------------+------------------+------------------+



In [8]:
df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1| 2053|
|      0| 4313|
+-------+-----+



In [9]:
df.groupBy('rate_marriage', 'affairs').count().show()

+-------------+-------+-----+
|rate_marriage|affairs|count|
+-------------+-------+-----+
|            1|      0|   25|
|            3|      1|  547|
|            4|      0| 1518|
|            1|      1|   74|
|            2|      1|  221|
|            2|      0|  127|
|            5|      0| 2197|
|            5|      1|  487|
|            3|      0|  446|
|            4|      1|  724|
+-------------+-------+-----+



In [10]:
# 婚姻评分与婚姻关系
df.groupBy('rate_marriage', 'affairs').count().orderBy('rate_marriage', 'affairs', 'count', ascending=True).show()

+-------------+-------+-----+
|rate_marriage|affairs|count|
+-------------+-------+-----+
|            1|      0|   25|
|            1|      1|   74|
|            2|      0|  127|
|            2|      1|  221|
|            3|      0|  446|
|            3|      1|  547|
|            4|      0| 1518|
|            4|      1|  724|
|            5|      0| 2197|
|            5|      1|  487|
+-------------+-------+-----+



In [11]:
# 宗教信仰与婚姻关系
df.groupBy('religious', 'affairs').count().orderBy('religious', 'affairs', 'count', ascending=True).show()

+---------+-------+-----+
|religious|affairs|count|
+---------+-------+-----+
|        1|      0|  613|
|        1|      1|  408|
|        2|      0| 1448|
|        2|      1|  819|
|        3|      0| 1715|
|        3|      1|  707|
|        4|      0|  537|
|        4|      1|  119|
+---------+-------+-----+



In [12]:
# 子女数量与婚姻关系
df.groupBy('children', 'affairs').count().orderBy('children', 'affairs', 'count', ascending=True).show()


+--------+-------+-----+
|children|affairs|count|
+--------+-------+-----+
|     0.0|      0| 1912|
|     0.0|      1|  502|
|     1.0|      0|  747|
|     1.0|      1|  412|
|     2.0|      0|  873|
|     2.0|      1|  608|
|     3.0|      0|  460|
|     3.0|      1|  321|
|     4.0|      0|  197|
|     4.0|      1|  131|
|     5.5|      0|  124|
|     5.5|      1|   79|
+--------+-------+-----+



In [13]:
df.groupBy('affairs').mean().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------+
|affairs|avg(rate_marriage)|          avg(age)|  avg(yrs_married)|     avg(children)|    avg(religious)|avg(affairs)|
+-------+------------------+------------------+------------------+------------------+------------------+------------+
|      1|3.6473453482708234|30.537018996590355|11.152459814905017|1.7289332683877252| 2.261568436434486|         1.0|
|      0| 4.329700904242986| 28.39067934152562| 7.989334569904939|1.2388128912589844|2.5045212149316023|         0.0|
+-------+------------------+------------------+------------------+------------------+------------------+------------+



In [14]:
# 步骤4： 特征工程
from pyspark.ml.feature import VectorAssembler
df_assembler = VectorAssembler(inputCols=['rate_marriage', 'age', 'yrs_married', 'children', 'religious'], outputCol="features")

In [15]:
df=df_assembler.transform(df)
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)
 |-- features: vector (nullable = true)



In [16]:
df.select(['features', 'affairs']).show(10, False)

+-----------------------+-------+
|features               |affairs|
+-----------------------+-------+
|[5.0,32.0,6.0,1.0,3.0] |0      |
|[4.0,22.0,2.5,0.0,2.0] |0      |
|[3.0,32.0,9.0,3.0,3.0] |1      |
|[3.0,27.0,13.0,3.0,1.0]|1      |
|[4.0,22.0,2.5,0.0,1.0] |1      |
|[4.0,37.0,16.5,4.0,3.0]|1      |
|[5.0,27.0,9.0,1.0,1.0] |1      |
|[4.0,27.0,9.0,0.0,2.0] |1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
+-----------------------+-------+
only showing top 10 rows



In [17]:
# 步骤5: 划分数据集
model_df = df.select(['features', 'affairs'])

In [18]:
train_df, test_df = model_df.randomSplit([0.75, 0.25])
print(train_df.count())

4741


In [19]:
train_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1| 1513|
|      0| 3228|
+-------+-----+



In [20]:
test_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1|  540|
|      0| 1085|
+-------+-----+



In [22]:
# 步骤6: 构建和训练随机森林模型
from pyspark.ml.classification import RandomForestClassifier
rf_classifier = RandomForestClassifier(labelCol='affairs', numTrees=50).fit(train_df)

In [23]:
# 步骤7: 基于测试数据进行评估
rf_predictions = rf_classifier.transform(test_df)
rf_predictions.show()
# 输出格式：
# 特征 真实标签 置信度 类别条件概率 预测标签

+--------------------+-------+--------------------+--------------------+----------+
|            features|affairs|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,22.0,2.5,0.0...|      1|[18.4106178655330...|[0.36821235731066...|       1.0|
|[1.0,22.0,2.5,0.0...|      1|[18.4106178655330...|[0.36821235731066...|       1.0|
|[1.0,22.0,2.5,1.0...|      1|[19.9617822479001...|[0.39923564495800...|       1.0|
|[1.0,22.0,2.5,1.0...|      0|[20.9029330494226...|[0.41805866098845...|       1.0|
|[1.0,27.0,2.5,0.0...|      0|[18.2780513051927...|[0.36556102610385...|       1.0|
|[1.0,27.0,6.0,1.0...|      1|[22.5683015169502...|[0.45136603033900...|       1.0|
|[1.0,27.0,6.0,1.0...|      0|[19.5529587147099...|[0.39105917429419...|       1.0|
|[1.0,27.0,6.0,2.0...|      1|[17.9958017209918...|[0.35991603441983...|       1.0|
|[1.0,27.0,9.0,2.0...|      1|[15.1388539257283...|[0.30277707851456...|    

In [24]:
rf_predictions.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 1331|
|       1.0|  294|
+----------+-----+



In [25]:
# 评估预测，引入类
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [29]:
# 准确率
rf_accuracy = MulticlassClassificationEvaluator(labelCol='affairs', metricName='accuracy').evaluate(rf_predictions)
print(rf_accuracy)
print('The assuary of RF on test data is {0:.0%}'.format(rf_accuracy))

0.7156923076923077
The assuary of RF on test data is 72%


In [33]:
# 精度（其实是加权精度，不能直接用的，书中错误）
# https://blog.csdn.net/Wang_xPP/article/details/104615416
# https://blog.csdn.net/qq_32531519/article/details/106242693
rf_precision = MulticlassClassificationEvaluator(labelCol='affairs', metricName='weightedPrecision').evaluate(rf_predictions)
print('The precision of RF on test data is {0:.0%}'.format(rf_precision))

The precision of RF on test data is 70%


In [35]:
# AUC曲线下的面积
rf_auc=BinaryClassificationEvaluator(labelCol='affairs').evaluate(rf_predictions)
print(rf_auc)

0.7426745178358082


In [36]:
rf_classifier.featureImportances

SparseVector(5, {0: 0.6295, 1: 0.027, 2: 0.2195, 3: 0.0728, 4: 0.0513})

In [37]:
# 了解哪个输入特征被影射到了哪个索引值
df.schema['features'].metadata['ml_attr']['attrs']

{'numeric': [{'idx': 0, 'name': 'rate_marriage'},
  {'idx': 1, 'name': 'age'},
  {'idx': 2, 'name': 'yrs_married'},
  {'idx': 3, 'name': 'children'},
  {'idx': 4, 'name': 'religious'}]}

In [38]:
# 步骤8 保存模型
from pyspark.ml.classification import RandomForestClassificationModel
rf_classifier.save("/Users/zhurunlong/Documents/output/rf_model")

In [39]:
rf=RandomForestClassificationModel.load('/Users/zhurunlong/Documents/output/rf_model')

In [ ]:
new_preditions = rf.transform(new_df)